# Skeleton drawing

In [49]:
%matplotlib inline
import pandas as pd
import cv2
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os.path
import json
import tqdm
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

#### Set annotation file path

In [26]:
annFolder = "../annotations"
annFile = "person_keypoints_val2017.json"
annPath = "{}/{}".format(annFolder, annFile)
print(annPath)
os.path.isfile(annPath)

../annotations/person_keypoints_val2017.json


True

#### Initialize COCO class for annotation

In [27]:
coco=COCO(annPath)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


#### Load input images from file

In [28]:
df = pd.read_csv("./inputImages.csv")
df

,imageID,filename
0,3156,000000003156.jpg
1,5529,000000005529.jpg
2,9448,000000009448.jpg
3,9483,000000009483.jpg
4,10764,000000010764.jpg
...,...,...
249,568584,000000568584.jpg
250,573258,000000573258.jpg
251,578545,000000578545.jpg
252,578871,000000578871.jpg


In [29]:
with open("./detectedPose.json") as jsonFile:
    detectedPose = json.load(jsonFile)
    jsonFile.close();
          
#detectedPose

In [44]:
lineConnection = [
    (3, 1), # Left ear to Left eye
    (4, 2), # Right ear to Right eye
    (1, 0), # Left eye to Nose
    (2, 0), # Right eye to Nose
    (5, 6), # Left shoulder to Right shoulder
    (5, 11), # Left shoulder to Left hip
    (6, 12), # Right shoulder to Right hip
    (11, 12), # Left hip to Right hip
    (9, 7), # Left wrist to Left elbow
    (7, 5), # Left elbow to Left shoulder
    (10, 8), # Right wrist to Right elbow
    (8, 6), # Right elbow to Right shoulder
    (11, 13), # Left hip to Left knee
    (13, 15), # Left knee to Left ankle
    (12, 14), # Right hip to Right knee
    (14, 16), # Right knee to Right ankle
]

def drawCircle(image, point, color=(255, 0, 0)):
    return cv2.circle(image, point, 1, color, thickness=5)

def isValidPoint(point):
    return point[0] > 0 and point[1] > 0

def drawLine(image, fromPoint, toPoint, color=(0, 255, 0)):
    if isValidPoint(fromPoint) and isValidPoint(toPoint):        
        return cv2.line(image, fromPoint, toPoint, color, thickness=2)
    return image
    
def drawKeypointOnImage(image, keypoints, circleColor=(255, 0, 0), lineColor=(0, 255, 0)):
    xs = keypoints[0::3]
    ys = keypoints[1::3]
    
    for idx in range(len(lineConnection)):           
        fromIdx = lineConnection[idx][0]
        toIdx = lineConnection[idx][1]
        
        try:
            fromPoint = (xs[fromIdx], ys[fromIdx])
            toPoint = (xs[toIdx], ys[toIdx])        
            image = drawLine(image, fromPoint, toPoint, lineColor)            
        except:
            continue
    
    try:
        nosePoint = (xs[0], ys[0])
        midShoulderPoint = (int((xs[5] + xs[6])/2), int((ys[5] + ys[6])/2))
        image = drawLine(image, nosePoint, midShoulderPoint, lineColor)        
    except:
        pass
        
    for idx in range(len(xs)):
        image = drawCircle(image, (xs[idx], ys[idx]), circleColor)        
    
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   

In [31]:
def getFilename(imageID):
    return f"{imageID}.jpg".zfill(16)

In [46]:
def drawCocoKeypointOnImage(image, keypoints):    
    circleColor = (0, 255, 0)
    lineColor = (0, 0, 255)
    return drawKeypointOnImage(image, keypoints, circleColor=circleColor, lineColor=lineColor)

def drawMediapipeKeypointOnImage(image, keypoints):
    circleColor = (255, 0, 0)
    lineColor = (0, 255, 0)
    return drawKeypointOnImage(image, keypoints, circleColor=circleColor, lineColor=lineColor)


In [51]:
fileDirectory = "../images/mediapipeEvaluation"
for pose in tqdm.tqdm(detectedPose):
    imageID = pose["image_id"]
    filename = getFilename(imageID)    
        
    image = cv2.imread('../images/val2017/'+filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    annIds = coco.getAnnIds(imgIds=imageID)
    anns = coco.loadAnns(annIds)
        
    annotatedImage = drawCocoKeypointOnImage(image.copy(), anns[0]['keypoints'])        
    filepathToSave = f"{fileDirectory}/{filename.replace('.jpg','_coco.jpg')}"   
    cv2.imwrite(filepathToSave, annotatedImage) 
    
    annotatedImage = drawMediapipeKeypointOnImage(image.copy(), pose["keypoints"])  
    filepathToSave = f"{fileDirectory}/{filename.replace('.jpg','_mp.jpg')}"        
    cv2.imwrite(filepathToSave, annotatedImage) 
    
    

100%|█████████████████████████████████████████| 254/254 [00:04<00:00, 55.66it/s]
